# estimation of attraction coefficient
Tim Tyree<br>
10.7.2022

In [63]:
from IPython.display import clear_output
clear_output(wait=True)

import sys,os
og_stdout=sys.stdout
from lib.my_initialization import *

os.chdir(nb_dir)
from lib import *

# import cupy, cudf
# from lib.rapids_func import *

import seaborn as sns
import matplotlib.ticker as mtick

import warnings
warnings.simplefilter("ignore", UserWarning)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
# !pwd

In [70]:
from lib.model.attraction_sim_anneal_fit import comp_square_error_msr_aff_osc_period_fixed

In [2]:
# from scipy.optimize import minimize
from scipy.optimize import dual_annealing

In [4]:
plt.close()
#reset matplotlib settings
import matplotlib as mpl
sns.reset_orig()
mpl.rc_file_defaults()

# define module

# TODO: back of envelop estimation of amax

In [14]:
#TODO: dev run #30 (L=5) (and #31 (L=10)) using the following:
###########################
# mean min lifetime: Gamma
###########################
tscale=1e-3 #s per ms
lifetime_fk= 105.3 # (1.6) ms
lifetime_lr= 33.37 # (73) ms
Gamma_fk = lifetime_fk*tscale
Gamma_lr = lifetime_lr*tscale

###########################
# rms range: Gamma
###########################
R_rms_fk = 0.67 # ± 0.06 cm
R_rms_lr = 0.86 # ± 0.07 cm

######################################################
# max attraction coefficient: amax (Method 1, b.o.e.)
######################################################
mean_squared_distance_fk = R_rms_fk**2
mean_squared_distance_lr = R_rms_lr**2
alinear_boe_fk = (mean_squared_distance_fk / Gamma_fk) / 4
alinear_boe_lr = (mean_squared_distance_lr / Gamma_lr) / 4
alinear_boe_fk,alinear_boe_lr

(1.0657644824311492, 5.540905004495055)

In [17]:
############################
# mean rotational period: T
############################
# #src: http://localhost:8889/notebooks/computing%20rotational%20period%20of%20mean%20fortranic%20spiral%20trajectories.ipynb
# - LR: mean number of loops per particle lifetime: 0.3438
# - FK: mean number of loops per particle lifetime: 0.4976
# - FK(backup, failing ^that): mean number of loops per particle lifetime (at least 1 loop): 0.9131
T_fk = Gamma_fk / 0.4976 #0.9131
T_lr = Gamma_lr / 0.3438
print(f"{T_fk=:.4f} s")
print(f"{T_lr=:.4f} s")

T_fk=0.2116 s
T_lr=0.0971 s


In [60]:
#print back of envelope estimates for alinear for each of the estimates of the mean squared distance between spiral tips
######################################################
# max attraction coefficient: amax (Method 1, b.o.e.)
######################################################

mean_squared_distance_fk =  0.9022**2
mean_squared_distance_lr = 1.0733**2
# mean_squared_distance_fk =  0.5263**2
# mean_squared_distance_lr = 0.6635**2
# mean_squared_distance_fk =  1.1521
# mean_squared_distance_lr = 1.6040
# mean_squared_distance_fk = R_rms_fk**2
# mean_squared_distance_lr = R_rms_lr**2
alinear_boe_fk = (mean_squared_distance_fk / Gamma_fk) / 4
alinear_boe_lr = (mean_squared_distance_lr / Gamma_lr) / 4
print(f"{alinear_boe_fk=:.4f} cm^2/s")
print(f"{alinear_boe_lr=:.4f} cm^2/s")

alinear_boe_fk=1.9325 cm^2/s
alinear_boe_lr=8.6303 cm^2/s


# TODO(later): estimate alinear for fixed period

In [33]:
#####################
# Fenton-Karma
#####################
range_data_dir_lst=[
    '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_fk/200x200/tippos_per_c_001_annihilation_range_timeseries.pkl',
    '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/200x200/tippos_per_001_annihilation_range_timeseries.pkl',    
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_fk/200x200/tippos_per_c_001_annihilation_range_timeseries.pkl',
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/200x200/tippos_per_001_annihilation_range_timeseries.pkl',    
]

# #####################
# # Luo-Rudy
# #####################
#runtime ~13 minutes
# range_data_dir_lst=[
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/200x200/tippos_per_001_annihilation_range_timeseries.pkl',
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/250x250/tippos_per_c_001_annihilation_range_timeseries.pkl',
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/300x300/tippos_per_c_001_annihilation_range_timeseries.pkl',
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/350x350/tippos_per_c_001_annihilation_range_timeseries.pkl',
#     '/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/400x400/tippos_per_c_001_annihilation_range_timeseries.pkl',
#     #'/home/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/500x500/tippos_per_001_annihilation_range_timeseries.pkl',
#     #^this one is too big to track in a reasonable amount of time...     
# ]
for fn in range_data_dir_lst:
    print(f"{fn}")
    assert os.path.exists(fn)
print(f"^those exist")

/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_fk/200x200/tippos_per_c_001_annihilation_range_timeseries.pkl
/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/200x200/tippos_per_001_annihilation_range_timeseries.pkl
^those exist


In [96]:
#HINT: sim. anneal routine that calculates amax unconstrained
# max_tdeath=100. #ms #LR
max_tdeath=300. #ms FK
printing=True
D=0
use_sim_anneal=True
repeat_w_10X=True

In [97]:
#DONE: recall rotational period, T in seconds
max_tdeath_lst=[300.,100.,] #ms
T_lst=[T_fk,T_lr] #s

msr_dir_bootstrap_fk = '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_fk/200x200/tippos_per_c_001_annihilation_msr.csv'
msr_dir_bootstrap_lr = '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_lr/200x200/tippos_per_001_annihilation_msr.csv'
msr_dir_bootstrap_lst=[msr_dir_bootstrap_fk,msr_dir_bootstrap_lr]
print(f"printing for two file sets: FK and then LR...")
# for max_tdeath,T,range_data_dir in zip(reversed(max_tdeath_lst),reversed(T_lst),reversed(range_data_dir_lst)):
for max_tdeath,T,range_data_dir,msr_dir_bootstrap in zip(max_tdeath_lst,T_lst,range_data_dir_lst,msr_dir_bootstrap_lst):
    #DONE: dev sim. anneal routine that calculates amax constrained to T.
    # #DONE: perform simulated annealing
    # #input: T, Gamma, MSR(t'), D=0
    # #output: mean squared distance
    # for range_data_dir in range_data_dir_lst:
    print(f"\n{range_data_dir=}")
    data_lst=load_from_pickle(range_data_dir)
    num_termination_events = len(data_lst)
    print(f"loaded range timeseries from {len(data_lst)} termination events")

    df=pd.concat(data_lst)
    del data_lst
    print(f"{df.shape=}")
    # compute mean minimum lifetime
    minlifetime_values = df.groupby(by=['trial_num','annihilation_index'])['tdeath'].max().values
    meanx,Delta_meanx,num_obsx,p_normalx=comp_mean_bootstrap_uncertainty(minlifetime_values)
    printing=True
    if printing:
        print(f" - o - mean lifetime: {meanx:.4f} +/- {Delta_meanx:.4f} ms (N={num_obsx}, p_normal: {p_normalx:.4f}) from {num_termination_events} termination events")
    Gamma=float(meanx)
    meanrange_values = df.groupby(by=['trial_num','annihilation_index'])['R'].mean().values
    meanR,Delta_meanR,num_obsR,p_normalR=comp_mean_bootstrap_uncertainty(meanrange_values)
    printing=True
    if printing:
        print(f"averages for spiral tip pairs:")
        print(f" - x - particle average first: mean range: {meanR:.4f} +/- {Delta_meanR:.4f} ms (N={num_obsR}, p_normal: {p_normalR:.4f}) from {num_termination_events} termination events")
        #print(f" - x - particle average first: mean range: {meanR:.4f} +/- {Delta_meanR:.4f} ms (N={num_obsR}, p_normal: {p_normalR:.4f}) from {num_termination_events} termination events")
    #     print(f" - x - particle average first (not this one!): mean range: {meanR:.4f} +/- {Delta_meanR:.4f} ms (N={num_obsR}, p_normal: {p_normalR:.4f}) from {num_termination_events} termination events")
    meanrange_values = df['R'].values
    meanR,Delta_meanR,num_obsR,p_normalR=comp_mean_bootstrap_uncertainty(meanrange_values)
    if printing:
        print(f" - o - all positions together (use this one!): mean range: {meanR:.4f} +/- {Delta_meanR:.4f} ms (N={num_obsR}, p_normal: {p_normalR:.4f}) from {num_termination_events} termination events")
    meanSR,Delta_meanSR,num_obsSR,p_normalSR=comp_mean_bootstrap_uncertainty(meanrange_values**2)
    if printing:
        print(f" - o - all positions together (use this one!): root mean squared range: {np.sqrt(meanSR):.4f} +/- {0.5*np.sqrt(Delta_meanSR)/np.sqrt(meanSR):.4f} ms (N={num_obsSR}, p_normal: {p_normalSR:.4f}) from {num_termination_events} termination events")
        print(f" - o - all positions together (use this one!): mean squared range: {meanSR:.4f} +/- {Delta_meanSR:.4f} ms (N={num_obsSR}, p_normal: {p_normalSR:.4f}) from {num_termination_events} termination events")
    if printing:
        print(f"loading bootstrapped mean...")
    df_bootstrap = pd.read_csv(msr_dir_bootstrap)
    x_values=df_bootstrap['tdeath_avg'].values
    y_values=df_bootstrap['MSR'].values
    yerr_values=df_bootstrap['Delta_MSR'].values
    x_values.shape,y_values.shape,yerr_values.shape
    # optionally fit alinear with simulated annealing
    # select msr from the full models for fitting
    boo=x_values<max_tdeath
    t_values=x_values[boo].copy()*1e-3
    msr_values=y_values[boo].copy() 
    # msr_values-= np.min(msr_values) #rmse = 0.58  with this uncommented
    #rmse = 0.66  with zero not subtracted.  also alinear was larger for LR model
    maxiter=10000
    print(f"running simulated annealing for {maxiter} epochs...")
    # bounds = ((0, 100), (0, 100), (50, 300), (-3.15,3.15))
    bounds = ((0, 1.), (0, 10000.), (-3.15,3.15))
    # msr_dir_bootstrap = msr_dir_bootstrap_lr
    print(f"{max_tdeath=} ms")
    print(f"src: {msr_dir_bootstrap=}")
    if use_sim_anneal:
        maxiter=10000
        print(f"num. epochs: {maxiter}")
        #x0=a0,a1,phase
        period=T*1e3 #bc period is in milliseconds
        args=t_values,msr_values,D,period
        res = dual_annealing(comp_square_error_msr_aff_osc_period_fixed,
                    bounds=bounds,
                    args=args,
                   maxiter=maxiter,seed=42,
                   no_local_search=True)#,**kwargs)

        rmse= np.sqrt(res.fun/t_values.shape[0])
        a0,a1,phase=res.x
        print(f"simulated annealing fit: {a0=:.9f}, {a1=:.9f}, {period=:.9f}, {phase=:.9f} --> {rmse=:.9f} cm^2")

        alinear = comp_alinear(a0,a1,period,Gamma,phase)
        omega = 2*np.pi/period
        z = Gamma*omega
        print(f"\tattraction coefficient: {a0=:.4f}, {a1=:.4f}, {z=:.4f}, {phase=:.4f} --> {alinear=:.4f} cm^2/s")
        print(f">>> {alinear=} \n")
        #print(f"*************** end data set ********************")

        if repeat_w_10X:
            #optionally repeat at 10X number of epochs
            maxiter=100000
            print(f"running simulated annealing for {maxiter} epochs...")
            #x0=a0,a1,phase
            #period=T*1e3 #bc period is in milliseconds
            #args=t_values,msr_values,D,period
            res = dual_annealing(comp_square_error_msr_aff_osc_period_fixed,
                        bounds=bounds,
                        args=args,
                       maxiter=maxiter,seed=42,
                       no_local_search=True)#,**kwargs)

            rmse= np.sqrt(res.fun/t_values.shape[0])
            a0,a1,phase=res.x
            print(f"simulated annealing fit: {a0=:.9f}, {a1=:.9f}, {period=:.9f}, {phase=:.9f} --> {rmse=:.9f} cm^2")

            alinear = comp_alinear(a0,a1,period,Gamma,phase)
            omega = 2*np.pi/period
            z = Gamma*omega
            print(f"\tattraction coefficient: {a0=:.4f}, {a1=:.4f}, {z=:.4f}, {phase=:.4f} --> {alinear=:.4f} cm^2/s")
            print(f">>> {alinear=} \n")
            print(f"*************** end data set ********************")

printing for two file sets: FK and then LR...

range_data_dir='/Users/timothytyree/Documents/GitHub/care/notebooks/Data/from_wjr/positions_fk/200x200/tippos_per_c_001_annihilation_range_timeseries.pkl'
loaded range timeseries from 80 termination events
df.shape=(3846532, 9)
 - o - mean lifetime: 105.3196 +/- 1.6647 ms (N=36179, p_normal: 0.3357) from 80 termination events
averages for spiral tip pairs:
 - x - particle average first: mean range: 0.5263 +/- 0.0034 ms (N=36179, p_normal: 0.9875) from 80 termination events
 - x - particle average first: mean range: 0.5263 +/- 0.0034 ms (N=36179, p_normal: 0.9875) from 80 termination events
 - o - all positions together (use this one!): mean range: 0.9022 +/- 0.0006 ms (N=3846532, p_normal: 0.2255) from 80 termination events
 - o - all positions together (use this one!): root mean squared range: 1.0734 +/- 0.0182 ms (N=3846532, p_normal: 0.3241) from 80 termination events
 - o - all positions together (use this one!): mean squared range: 1.

In [ ]:
>>> alinear=1.8402023397115908 
>>> alinear=8.585030312541999 


In [98]:
alinear_anneal_fk = 1.8402023397115908 
alinear_anneal_lr = 8.585030312541999 

In [99]:
# x0=a0,a1,phase
# period=T*1e3 #bc period is in milliseconds
# args=t_values,msr_values,D,period
# square_error=comp_square_error_msr_aff_osc_period_fixed(x0,*args)
# rmse=np.sqrt(square_error/t_values.shape[0])
# print(f"{rmse=}")

In [100]:
# ######################################################
# # TODO: max attraction coefficient: amax (Method 2, annealed)
# ######################################################
# mean_squared_distance_hat_fk = ??
# mean_squared_distance_hat_lr = ??
# alinear_anneal_fk = mean_squared_distance_hat_fk / Gamma_fk / 4
# alinear_anneal_lr = mean_squared_distance_hat_lr / Gamma_lr / 4
#TODO: compare ibid to 
alinear_boe_fk = mean_squared_distance_fk / Gamma_fk / 4
alinear_boe_lr = mean_squared_distance_lr / Gamma_lr / 4
print(f"{alinear_anneal_fk=} cm^2/s")
print(f"{alinear_boe_fk=} cm^2/s")
print(f"{alinear_anneal_lr=} cm^2/s")
print(f"{alinear_boe_lr=} cm^2/s")

alinear_anneal_fk=1.8402023397115908 cm^2/s
alinear_boe_fk=1.9324901234567902 cm^2/s
alinear_anneal_lr=8.585030312541999 cm^2/s
alinear_boe_lr=8.630303341324543 cm^2/s


In [20]:
#TODO: dev next data run #30!
#TODO: play in midas for the remainder of the evening...

In [23]:
# ######################################################
# # diffusion coefficient: D (from time averaged MSD)
# ######################################################
# $D$ (cm$^2$/s):
# - FK: 37.2$\pm$1.4$\times10^{-3}$
# - LR: 1.40$\pm$0.10$\times10^{-3}$
# - FK(failing ^that, from MSR): D = m/8 = 0.1145 +/- 0.007987364738330576 (using timepoints after 600.0 ms)

# ######################################################
# # reaction rate: kappa (b.o.e. from death rates and qs distribution)
# ######################################################
# - Area: 25 cm^2, kappa: 436.74 Hz (Fenton-Karma)
# - Area: 25 cm^2, kappa: 1614.16 Hz (Luo-Rudy)
# - Area: 100 cm^2, kappa: 6197.13 Hz (Fenton-Karma)
# - Area: 100 cm^2, kappa: 8532.76 Hz (Luo-Rudy)

# then, TODO: dev run #30 using the above parameter values

In [101]:
# # # Compute MSR(t') by averaging range timeseries
# # x_values_in=df['tdeath'].values
# # y_values_in=df['R'].values**2
# # print(f"tdeath ranges from {np.min(x_values_in)} to {np.max(x_values_in)} ms")
# # print(f"R^2 ranges from {np.min(y_values_in)} to {np.max(y_values_in)} cm^2")

# # assert not np.isnan(x_values_in).any() 
# # assert not np.isnan(y_values_in).any()

# # #compute bootstrap 
# # printing=True
# # if printing:
# #     print(f"loading bootstrapped mean...")
# # #     print(f"bootstrapping mean (estimated run time ~30 seconds)...")
# # # bins=np.arange(0,450,1)
# # # bins=np.arange(0,1800,5)
# # bins=np.arange(0,2800,5)
# # df_bootstrap=bin_and_bootstrap_xy_values_parallel(x_values_in,
# #                                y_values_in,
# #                                xlabel='tdeath_avg',
# #                                ylabel='MSR',
# #                                bins=bins,
# #                                min_numobs=10,
# #                                num_bootstrap_samples=1000,
# #                                npartitions=10,
# #                                use_test=False)
# # #save msr as .csv
# # msr_dir_bootstrap=range_data_dir.replace('_range_timeseries.pkl','_msr.csv')
# # df_bootstrap.to_csv(msr_dir_bootstrap,index=False)
# # print(f"{msr_dir_bootstrap=}")

# # if printing:
# #     print(f"loading bootstrapped mean...")
# df_bootstrap = pd.load_csv(msr_dir_bootstrap)

# x_values=df_bootstrap['tdeath_avg'].values
# y_values=df_bootstrap['MSR'].values
# yerr_values=df_bootstrap['Delta_MSR'].values
# x_values.shape,y_values.shape,yerr_values.shape


# # optionally fit alinear with simulated annealing
# # select msr from the full models for fitting
# boo=x_values<max_tdeath
# t_values=x_values[boo].copy()*1e-3
# msr_values=y_values[boo].copy() 
# msr_values-= np.min(msr_values)
# print(f"{max_tdeath=} ms")
# print(f"src: {msr_dir_bootstrap=}")
# if use_sim_anneal:
#     maxiter=10000
#     print(f"num. epochs: {maxiter}")
#     res = anneal_msr_fit(t_values,msr_values,D,
#                        bounds = ((0, 100), (0, 100), (50, 300), (-3.15,3.15)),
#                        maxiter=maxiter,seed=42, no_local_search=True)#,**kwargs)
#     rmse= np.sqrt(res.fun/t_values.shape[0])
#     a0,a1,period,phase=res.x
#     #print(f"simulated annealing fit: {a0=:.4f}, {a1=:.4f}, {period=:.4f}, {phase=:.4f} --> {rmse=:.4f} cm^2")

#     alinear = comp_alinear(a0,a1,period,Gamma,phase)
#     omega = 2*np.pi/period
#     z = Gamma*omega
#     print(f"\tsimulated annealing fit: {a0=:.9f}, {a1=:.9f}, {z=:.9f}, {phase=:.9f} --> {alinear=:.9f} cm^2/s")
#     print(f">>> {alinear=} \n")    
#     if repeat_w_10X:
#         #optionally repeat at 10X number of epochs
#         maxiter=100000
#         print(f"running simulated annealing for {maxiter} epochs...")
#         bounds = ((0, 100), (0, 100), (50, 300), (-3.15,3.15))
        
#         #x0=a0,a1,phase
#         period=T*1e3 #bc period is in milliseconds
#         args=t_values,msr_values,D,period
#         res = dual_annealing(comp_square_error_msr_aff_osc
#                    ,
#                    maxiter=maxiter,seed=42,
#                    no_local_search=True)#,**kwargs)
        
            
#     # np.random.seed(42)   # seed to allow replication.
#     res = dual_annealing(
#         comp_square_error_msr_aff_osc,
#         bounds=bounds,
#         args=args,
#         maxiter=maxiter,
#         seed=seed,
#         no_local_search=no_local_search,**kwargs)
        
        
        
        
        
        
        
#         res = anneal_msr_fit(t_values,msr_values,D,
#                            bounds = ((0, 100), (0, 100), (50, 300), (-3.15,3.15)),
#                            maxiter=maxiter,seed=42+1, no_local_search=True)#,**kwargs)
#         rmse= np.sqrt(res.fun/t_values.shape[0])
#         a0,a1,period,phase=res.x
#         print(f"simulated annealing fit: {a0=:.9f}, {a1=:.9f}, {period=:.9f}, {phase=:.9f} --> {rmse=:.9f} cm^2")

#         alinear = comp_alinear(a0,a1,period,Gamma,phase)
#         omega = 2*np.pi/period
#         z = Gamma*omega
#         print(f"\tattraction coefficient: {a0=:.4f}, {a1=:.4f}, {z=:.4f}, {phase=:.4f} --> {alinear=:.4f} cm^2/s")
#         print(f">>> {alinear=} \n")
#         print(f"*************** end data set ********************")